Requirements

In [1]:
!pip install nnunetv2 tqdm s3fs
from tqdm import tqdm
import torch
import s3fs
from pathlib import Path
from tqdm import tqdm
import subprocess
import threading
import time
import os

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 86.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 82.6 MB/s eta 0:00:00:

Identifiers

In [ ]:
email = input("Enter your email ENSAE: ")
name = input("Enter your username Onyxia: ")

subprocess.run(["git", "config", "--global", "user.email", email])
subprocess.run(["git", "config", "--global", "user.name", name])

print(f"Git configured with email : {email} and username : {name}")

S3 private keys

In [ ]:
aws_access_key_id = input("Enter your AWS_ACCESS_KEY_ID: ")
aws_secret_access_key = input("Enter your AWS_SECRET_ACCESS_KEY: ")
aws_session_token = input("Enter your AWS_SESSION_TOKEN: ")

# Environment variables
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
os.environ["AWS_SESSION_TOKEN"] = aws_session_token

print("AWS keys configured as environment variables.")


Connection to S3

In [ ]:
# Connexion to  MinIO S3 Onyxia
s3 = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key=os.getenv("AWS_ACCESS_KEY_ID"),
    secret=os.getenv("AWS_SECRET_ACCESS_KEY"),
    token=os.getenv("AWS_SESSION_TOKEN")
)
#print(len(s3.ls("projet-statapp-segmedic/diffusion/nnunet_dataset/nnUNet_raw/Dataset001_Annot1/labelsTr")))

Download S3 data locally

In [ ]:
def download_s3_folder():
    
    # Defining paths
    base_local_path = Path('/tmp/nnunet')
    s3_base_path = "projet-statapp-segmedic/diffusion/nnunet_dataset"
    folders = ['nnUNet_raw', 'nnUNet_preprocessed', 'nnUNet_results']
    
    # Creating local folders
    for folder in folders:
        local_folder = base_local_path / folder
        local_folder.mkdir(parents=True, exist_ok=True)
        
        s3_path = f"{s3_base_path}/{folder}"
        print(f"\nTéléchargement du dossier {folder}...")
        
        # Recursive list of all files from S3
        try:
            files = s3.find(s3_path)
            
            # Progression bar (very nice!)
            with tqdm(total=len(files), desc=f"Fichiers dans {folder}") as pbar:
                for file_path in files:
                    relative_path = file_path.replace(s3_path, '').lstrip('/')
                    local_file_path = local_folder / relative_path
                    
                    # Creating local files if needed
                    local_file_path.parent.mkdir(parents=True, exist_ok=True)
                    
                    # Dowloading files
                    if not local_file_path.exists():
                        try:
                            s3.get(file_path, str(local_file_path))
                        except Exception as e:
                            print(f"Error while downloading {file_path}: {e}")
                    
                    pbar.update(1)
        
        except Exception as e:
            print(f"Error while reading {s3_path}: {e}")
            continue
        
        #ERROR CORRECTED: the nnU-Net dataset naming convention requires 4 digit for image case file, not 3. 
        for string in ['1', '2', '3']:
            images = Path(f"/tmp/nnunet/nnUNet_raw/Dataset00{string}_Annot{string}/imagesTr")
            for f in images.glob("*_000.nii.gz"):
                f.rename(f.with_name(f.name.replace("_000.nii.gz", "_0000.nii.gz")))
    
    # Creating global variables for paths, needed for nnU-Net training. 
    env_vars = {
        'nnUNet_raw': str(base_local_path / 'nnUNet_raw'),
        'nnUNet_preprocessed': str(base_local_path / 'nnUNet_preprocessed'),
        'nnUNet_results': str(base_local_path / 'nnUNet_results')
    }
    
    for var_name, path in env_vars.items():
        os.environ[var_name] = path
    
    # Adding to .bashrc
    with open(os.path.expanduser('~/.bashrc'), 'a') as f:
        f.write('\n# nnUNet paths\n')
        for var_name, path in env_vars.items():
            f.write(f'export {var_name}="{path}"\n')
    
    print("\nConfiguration finished. Environment variables created:")
    for var_name, path in env_vars.items():
        print(f"{var_name}={path}")

    #To apply changes:
    !source ~/.bashrc

download_s3_folder()